In [9]:
""" Real labels evaluator for ImageNet
Paper: `Are we done with ImageNet?` - https://arxiv.org/abs/2006.07159
Based on Numpy example at https://github.com/google-research/reassessed-imagenet

Hacked together by / Copyright 2020 Ross Wightman
"""
import os
import json
import numpy as np
import sys
sys.path.append("../")
from timm.data import create_dataset, create_loader, resolve_data_config


class RealLabelsImagenet:

    def __init__(self, filenames, real_json='../imagenet/real.json', topk=(1, 5)):
        with open(real_json) as real_labels:
            real_labels = json.load(real_labels)
            real_labels = {f'ILSVRC2012_val_{i + 1:08d}.JPEG': labels for i, labels in enumerate(real_labels)}
        self.real_labels = real_labels
        self.filenames = filenames
        assert len(self.filenames) == len(self.real_labels)
        self.topk = topk
        self.is_correct = {k: [] for k in topk}
        self.sample_idx = 0

    def add_result(self, output):
        maxk = max(self.topk)
        _, pred_batch = output.topk(maxk, 1, True, True)
        pred_batch = pred_batch.cpu().numpy()
        for pred in pred_batch:
            filename = self.filenames[self.sample_idx]
            filename = os.path.basename(filename)
            if self.real_labels[filename]:
                for k in self.topk:
                    self.is_correct[k].append(
                        any([p in self.real_labels[filename] for p in pred[:k]]))
            self.sample_idx += 1

    def get_accuracy(self, k=None):
        if k is None:
            return {k: float(np.mean(self.is_correct[k])) * 100 for k in self.topk}
        else:
            return float(np.mean(self.is_correct[k])) * 100


In [13]:
dataset = create_dataset(root="../imagenet/validation/", name="", split="validation", load_bytes=False, class_map="")

In [15]:
real_labels = RealLabelsImagenet(dataset.filenames(basename=True), real_json="../imagenet/real.json")

In [21]:
real_labels.get_accuracy

TypeError: 'method' object is not subscriptable

In [23]:
real_labels.real_labels

{'ILSVRC2012_val_00000001.JPEG': [],
 'ILSVRC2012_val_00000002.JPEG': [970, 795],
 'ILSVRC2012_val_00000003.JPEG': [230, 231],
 'ILSVRC2012_val_00000004.JPEG': [809],
 'ILSVRC2012_val_00000005.JPEG': [516, 850],
 'ILSVRC2012_val_00000006.JPEG': [57],
 'ILSVRC2012_val_00000007.JPEG': [334],
 'ILSVRC2012_val_00000008.JPEG': [700],
 'ILSVRC2012_val_00000009.JPEG': [674],
 'ILSVRC2012_val_00000010.JPEG': [332],
 'ILSVRC2012_val_00000011.JPEG': [109],
 'ILSVRC2012_val_00000012.JPEG': [286],
 'ILSVRC2012_val_00000013.JPEG': [370],
 'ILSVRC2012_val_00000014.JPEG': [757],
 'ILSVRC2012_val_00000015.JPEG': [595],
 'ILSVRC2012_val_00000016.JPEG': [147],
 'ILSVRC2012_val_00000017.JPEG': [327, 108],
 'ILSVRC2012_val_00000018.JPEG': [21, 22],
 'ILSVRC2012_val_00000019.JPEG': [478],
 'ILSVRC2012_val_00000020.JPEG': [517],
 'ILSVRC2012_val_00000021.JPEG': [334],
 'ILSVRC2012_val_00000022.JPEG': [],
 'ILSVRC2012_val_00000023.JPEG': [948],
 'ILSVRC2012_val_00000024.JPEG': [727],
 'ILSVRC2012_val_0000002

In [32]:
import csv

with open('../result/real_label.csv', 'w') as f:  
    writer = csv.writer(f)
    for k, v in real_labels.real_labels.items():
        data = [k]
        if len(v) == 0 :
            data.append(9999)
        for v_one in v:
            data.append(v_one)
        writer.writerow(data)

In [5]:
with open('imagenet1000_clsidx_to_labels.txt','r') as inf:
    dict_from_file = eval(inf.read())
dict_from_file

'tench, Tinca tinca'